# List of Exposures with Hologram


- work with Weakly_2022_09
- use jupyter kernel LSST



- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2022/03/22

- update : 2022/05/05
- last update : 2022/05/19 : check if 2022/05/04 where already ingested
- last update : 2022/06/16


In [1]:
! eups list -s | grep LOCAL

atmospec              LOCAL:/sps/lsst/groups/auxtel/softs/shared/auxteldm_gen3/repos/w_2022_09/atmospec 	setup
eups                  LOCAL:/cvmfs/sw.lsst.eu/linux-x86_64/lsst_distrib/w_2022_09/conda/envs/lsst-scipipe-2.0.0/eups 	setup
rapid_analysis        LOCAL:/sps/lsst/groups/auxtel/softs/shared/auxteldm_gen3/repos/w_2022_09/rapid_analysis 	setup


In [2]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib


   g64fc59b30a+e1dba0a6f6 	w_latest w_2022_09 current setup


In [3]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.io import fits

In [4]:
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [5]:
import lsst.daf.butler as dafButler

In [6]:
repo = '/sps/lsst/groups/auxtel/softs/shared/auxteldm_gen3/data/butler.yaml'
butler = dafButler.Butler(repo)
registry = butler.registry

## List of Exposures

In [7]:
df_exposure = pd.DataFrame(columns=['id', 'obs_id','day_obs', 'seq_num', 'type', 'target','filter','zenith_angle','expos','ra','dec','skyangle'])

In [8]:
for count, info in enumerate(registry.queryDimensionRecords('exposure')):
    
    
    df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num, info.observation_type, info.target_name, info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle]
    
    if count < 0:
        print("-----------------------------------------------------",count,"---------------------------------------------------------")
        print(info)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)
    

In [9]:
df_exposure

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
0,2021021600208,AT_O_20210216_000208,20210216,208,science,HD 50896,empty~empty,24.314073,2.0,103.384002,-23.973448,-180.0
1,2021021600163,AT_O_20210216_000163,20210216,163,science,HD 50896,RG610~empty,14.580611,3.0,103.404029,-23.92939,-208.621712
2,2021021600070,AT_O_20210216_000070,20210216,70,bias,azel_target,empty~empty,None,0.0,None,None,None
3,2021021600173,AT_O_20210216_000173,20210216,173,science,HD 50896,RG610~empty,8.82931,2.0,103.395986,-23.927038,-208.621712
4,2021021600001,AT_O_20210216_000001,20210216,1,bias,azel_target,empty~empty,None,0.0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
19070,2022060900838,AT_O_20220609_000838,20220609,838,science,spec:HD185975,BG40~holo4_003,57.194196,30.0,307.708009,-87.50686,105.70201
19071,2022060900839,AT_O_20220609_000839,20220609,839,science,spec:HD185975,FELH0600~holo4_003,57.196255,30.0,307.707897,-87.506868,105.701922
19072,2022060900840,AT_O_20220609_000840,20220609,840,science,spec:HD185975,FELH0600~holo4_003,57.198265,30.0,307.708264,-87.506854,105.701541
19073,2022060900841,AT_O_20220609_000841,20220609,841,science,spec:HD185975,FELH0600~holo4_003,57.200339,30.0,307.708109,-87.506873,105.701644


## Selection of science

In [10]:
df_science = df_exposure[df_exposure.type == 'science']

In [11]:
df_science.sort_values(by="id",ascending=True,inplace=True)

/tmp/ipykernel_6355/3986782663.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_science.sort_values(by="id",ascending=True,inplace=True)


In [12]:
df_science.tail(60)

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
18998,2022060900766,AT_O_20220609_000766,20220609,766,science,spec:HD205905,empty~ronchi170lpmm,19.548686,30.0,324.794559,-27.258511,346.341748
18999,2022060900767,AT_O_20220609_000767,20220609,767,science,spec:HD205905,empty~ronchi170lpmm,19.393817,30.0,324.794577,-27.258501,346.341711
19000,2022060900768,AT_O_20220609_000768,20220609,768,science,spec:HD205905,empty~ronchi170lpmm,19.238899,30.0,324.79452,-27.258501,346.341765
19001,2022060900769,AT_O_20220609_000769,20220609,769,science,spec:HD205905,empty~ronchi170lpmm,19.084132,30.0,324.794609,-27.258488,346.341744
19002,2022060900770,AT_O_20220609_000770,20220609,770,science,spec:HD205905,BG40~ronchi170lpmm,18.918483,30.0,324.794587,-27.258509,346.341739
19003,2022060900771,AT_O_20220609_000771,20220609,771,science,spec:HD205905,BG40~ronchi170lpmm,18.763642,30.0,324.794536,-27.258496,346.341758
19004,2022060900772,AT_O_20220609_000772,20220609,772,science,spec:HD205905,BG40~ronchi170lpmm,18.608877,30.0,324.794553,-27.258511,346.337373
19005,2022060900773,AT_O_20220609_000773,20220609,773,science,spec:HD205905,BG40~ronchi170lpmm,18.454103,30.0,324.79453,-27.258523,346.341696
19006,2022060900774,AT_O_20220609_000774,20220609,774,science,spec:HD205905,FELH0600~ronchi170lpmm,18.29219,30.0,324.794535,-27.258508,346.341701
19007,2022060900775,AT_O_20220609_000775,20220609,775,science,spec:HD205905,FELH0600~ronchi170lpmm,18.137542,30.0,324.79459,-27.258507,346.341849


In [13]:
df_science.day_obs.unique()

array([20210216, 20210217, 20210218, 20210311, 20210608, 20210609,
       20210610, 20210706, 20210707, 20210708, 20210907, 20210908,
       20210909, 20211005, 20211006, 20211102, 20211103, 20211104,
       20220215, 20220216, 20220217, 20220316, 20220317, 20220503,
       20220504, 20220524, 20220607, 20220608, 20220609])

## Selection of date

2021-02-16  2021-02-18  2021-06-08  2021-06-10  2021-07-07  2021-09-07  2021-09-09  2021-10-06  2021-11-02  2021-11-04  2022-02-16  2022-03-15  2022-03-17  20220502  
2021-02-17  2021-03-11  2021-06-09  2021-07-06  2021-07-08  2021-09-08  2021-10-05  2021-10-07  2021-11-03  2022-02-15  2022-02-17  2022-03-16  2022-03-18  20220503 20220524 20220607 20220608 20220609

### Add those

20220504 - Not existing

In [61]:
date_sel = 20220609

In [62]:
df_science_selected = df_science[df_science.day_obs == date_sel ]

In [63]:
len(df_science_selected)

644

# List of filters

In [64]:
list_of_filters = df_science_selected['filter'].unique()
list_of_filters

array(['SDSSg~empty', 'empty~holo4_003', 'BG40~holo4_003',
       'FELH0600~holo4_003', 'empty~ronchi170lpmm', 'BG40~ronchi170lpmm',
       'FELH0600~ronchi170lpmm'], dtype=object)

In [65]:
selected_filters = []
for filt in list_of_filters:
    #flag_sel = (filt.find('holo4') != -1) or (filt.find('ronchi90lpmm') != -1) or (filt.find('ronchi170lpmm') != -1)
    flag_sel = (filt.find('holo4') != -1) 
    #flag_sel = (filt.find('ronchi170') != -1) 
    if flag_sel:
        selected_filters.append(filt) 
selected_filters = np.array(selected_filters)
selected_filters       

array(['empty~holo4_003', 'BG40~holo4_003', 'FELH0600~holo4_003'],
      dtype='<U18')

## Selection of filter

- But now we have a loop on filters

In [66]:
#idx_sel_filt = 0
#selected_filter = selected_filters[0]
#selected_filter

## Final selection of exposure

- loop on selected filters

In [67]:
pd.set_option('display.max_rows', None)

In [68]:
all_def_science_selected_final = []

for selected_filter in selected_filters:

    print(selected_filter)
    df_science_selected_final = df_science_selected[df_science_selected["filter"] ==  selected_filter]
    
    #print(df_science_selected_final)
    all_def_science_selected_final.append(df_science_selected_final.copy())
    
    df = df_science_selected_final
    df_sel = df[['day_obs', 'seq_num']]
    
    
    NN = len(df)
    
    if NN>0:
        # build the line
        
        df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
    
    
         # prepare output
        filename_out = "visitdispersers_"+ str(date_sel) + "_filt_" + selected_filter.replace('~','-') + ".list"
        print(filename_out)
        
        df_out = df_sel[["line"]]
        df_out.to_csv(filename_out, index=False, header=None)

empty~holo4_003
visitdispersers_20220609_filt_empty-holo4_003.list
BG40~holo4_003
visitdispersers_20220609_filt_BG40-holo4_003.list
FELH0600~holo4_003
visitdispersers_20220609_filt_FELH0600-holo4_003.list


/tmp/ipykernel_6355/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
/tmp/ipykernel_6355/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
/tmp/ipykernel_6355/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

In [69]:
len(all_def_science_selected_final)

3

In [70]:
all_def_science_selected_final[0]

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
18311,2022060900079,AT_O_20220609_000079,20220609,79,science,spec:HD185975,empty~holo4_003,61.57148,30.0,306.492831,-87.440446,315.323923
18312,2022060900080,AT_O_20220609_000080,20220609,80,science,spec:HD185975,empty~holo4_003,61.565691,30.0,306.491316,-87.440421,315.32521
18313,2022060900081,AT_O_20220609_000081,20220609,81,science,spec:HD185975,empty~holo4_003,61.559908,30.0,306.488906,-87.440315,315.327437
18314,2022060900082,AT_O_20220609_000082,20220609,82,science,spec:HD185975,empty~holo4_003,61.554114,30.0,306.48969,-87.440352,315.326554
18334,2022060900102,AT_O_20220609_000102,20220609,102,science,spec:HD142331,empty~holo4_003,59.240782,30.0,238.567495,-8.5447,329.898169
18335,2022060900103,AT_O_20220609_000103,20220609,103,science,spec:HD142331,empty~holo4_003,59.087255,30.0,238.567451,-8.544736,329.898147
18336,2022060900104,AT_O_20220609_000104,20220609,104,science,spec:HD142331,empty~holo4_003,58.933944,30.0,238.567575,-8.544708,329.898325
18337,2022060900105,AT_O_20220609_000105,20220609,105,science,spec:HD142331,empty~holo4_003,58.780585,30.0,238.567595,-8.54468,329.898177
18361,2022060900129,AT_O_20220609_000129,20220609,129,science,spec:HD142331,empty~holo4_003,55.289224,30.0,238.566807,-8.545181,329.00674
18362,2022060900130,AT_O_20220609_000130,20220609,130,science,spec:HD142331,empty~holo4_003,55.13706,30.0,238.566852,-8.545142,329.006565


In [71]:
all_def_science_selected_final[1]

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
18315,2022060900083,AT_O_20220609_000083,20220609,83,science,spec:HD185975,BG40~holo4_003,61.547902,30.0,306.488094,-87.440265,315.327901
18316,2022060900084,AT_O_20220609_000084,20220609,84,science,spec:HD185975,BG40~holo4_003,61.542062,30.0,306.488269,-87.440284,315.327648
18317,2022060900085,AT_O_20220609_000085,20220609,85,science,spec:HD185975,BG40~holo4_003,61.536208,30.0,306.490367,-87.440391,315.325437
18318,2022060900086,AT_O_20220609_000086,20220609,86,science,spec:HD185975,BG40~holo4_003,61.530342,30.0,306.490799,-87.44041,315.324889
18338,2022060900106,AT_O_20220609_000106,20220609,106,science,spec:HD142331,BG40~holo4_003,58.612935,30.0,238.567543,-8.544667,329.898251
18339,2022060900107,AT_O_20220609_000107,20220609,107,science,spec:HD142331,BG40~holo4_003,58.459501,30.0,238.567418,-8.544716,329.898195
18340,2022060900108,AT_O_20220609_000108,20220609,108,science,spec:HD142331,BG40~holo4_003,58.306268,30.0,238.567454,-8.544724,329.89821
18341,2022060900109,AT_O_20220609_000109,20220609,109,science,spec:HD142331,BG40~holo4_003,58.15661,30.0,238.567457,-8.544737,329.894132
18365,2022060900133,AT_O_20220609_000133,20220609,133,science,spec:HD142331,BG40~holo4_003,54.67373,30.0,238.566922,-8.545147,329.006735
18366,2022060900134,AT_O_20220609_000134,20220609,134,science,spec:HD142331,BG40~holo4_003,54.521606,30.0,238.566797,-8.545215,329.006703


In [72]:
all_def_science_selected_final[2]

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
18319,2022060900087,AT_O_20220609_000087,20220609,87,science,spec:HD185975,FELH0600~holo4_003,61.524166,30.0,306.490629,-87.440425,315.325004
18320,2022060900088,AT_O_20220609_000088,20220609,88,science,spec:HD185975,FELH0600~holo4_003,61.518278,30.0,306.490105,-87.440374,315.325309
18321,2022060900089,AT_O_20220609_000089,20220609,89,science,spec:HD185975,FELH0600~holo4_003,61.512352,30.0,306.491428,-87.440446,315.324011
18322,2022060900090,AT_O_20220609_000090,20220609,90,science,spec:HD185975,FELH0600~holo4_003,61.506416,30.0,306.492248,-87.44048,315.323059
18342,2022060900110,AT_O_20220609_000110,20220609,110,science,spec:HD142331,FELH0600~holo4_003,57.996432,30.0,238.567574,-8.544699,329.893986
18343,2022060900111,AT_O_20220609_000111,20220609,111,science,spec:HD142331,FELH0600~holo4_003,57.843316,30.0,238.567582,-8.544691,329.894149
18344,2022060900112,AT_O_20220609_000112,20220609,112,science,spec:HD142331,FELH0600~holo4_003,57.690149,30.0,238.567506,-8.544733,329.893999
18345,2022060900113,AT_O_20220609_000113,20220609,113,science,spec:HD142331,FELH0600~holo4_003,57.537094,30.0,238.567489,-8.54474,329.894006
18369,2022060900137,AT_O_20220609_000137,20220609,137,science,spec:HD142331,FELH0600~holo4_003,54.059078,30.0,238.566919,-8.545198,329.002376
18370,2022060900138,AT_O_20220609_000138,20220609,138,science,spec:HD142331,FELH0600~holo4_003,53.907374,30.0,238.56693,-8.54516,329.002512


In [73]:
all_def_science_selected_final[3]

IndexError: list index out of range

In [ ]:
all_def_science_selected_final[4]